In [2]:
import pandas as pd
import numpy as np
from gurobipy import GRB
import gurobipy as gp

In [13]:
investments = [
    [0, 4.1, 5.8, 6.5, 6.8],
    [0, 1.8, 3.0, 3.9, 4.5],
    [0, 1.5, 2.5, 3.3, 3.8],
    [0, 2.2, 3.8, 4.8, 5.5],
    [0, 1.3, 2.4, 3.2, 3.9],
    [0, 4.2, 5.9, 6.6, 6.8],
    [0, 2.2, 3.5, 4.2, 4.6],
    [0, 1.0, 1.7, 2.3, 2.8]
]

In [4]:
m = gp.Model("portfolio")

Restricted license - for non-production use only - expires 2023-10-25


In [14]:
## Allocate decision variables
x = m.addMVar(shape=(len(investments)), vtype=GRB.INTEGER, name="decision")

In [15]:
## Add constraints on max/min investment levels
for i in range(x.shape[0]):
    m.addConstr(
        0 <= x[i]
    )
    m.addConstr(
        x[i] <= 4
    )

In [16]:
## Add constraint on total funding
m.addConstr(
    gp.quicksum(x[i]*10000000 for i in range(x.shape[0])) <= 100000000
)

<gurobi.Constr *Awaiting Model Update*>

In [22]:
## First part is revenue from investments, second part is return form savings
obj_f = sum([investments[i][x[i]] for i in range(x.shape[0])]) #+ (100000000 - sum([x[i]*10000000]))*0.05

TypeError: list indices must be integers or slices, not Var

In [23]:
m.setObjective(
    obj_f
    ,GRB.MAXIMIZE
)

NameError: name 'obj_f' is not defined

In [ ]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)